<a href="https://colab.research.google.com/github/NajwaaAzzahra/Machine_Learning/blob/main/Pertemuan%2010/Jobsheet_RNN_Praktikum2danTugas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Najwa Azzahra
#TI-3C/19
#2241720139

**Jobsheet Recurrent Neural Network Praktikum 2**

#Setup

In [1]:
#Import TensorFlow
import tensorflow as tf
import numpy as np
import os
import time

In [2]:
#Download Dataset Shakespeare
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
#Load Data
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print(f'Length of text: {len(text)} characters')

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

65 unique characters


#Olah Teks

In [6]:
#Vectorize Teks
example_texts = ['abcdefg', 'xyz']
chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [7]:
ids_from_chars = tf.keras.layers.StringLookup(
vocabulary=list(vocab), mask_token=None)

In [8]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [10]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [23]:
##Membuat Training Set dan Target
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [24]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [25]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

F
i
r
s
t
 
C
i
t
i


In [16]:
seq_length = 100

In [17]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


In [18]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [19]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [20]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [21]:
dataset = sequences.map(split_input_target)

In [22]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [26]:
#Membuat Batch Training
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

#Buat Model

In [31]:
# Length of the vocabulary in StringLookup Layer
vocab_size = len(ids_from_chars.get_vocabulary())

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [32]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super(MyModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = self.embedding(inputs)
        if states is None:
            states = self.gru.get_initial_state(batch_size=tf.shape(inputs)[0])
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x)

        if return_state:
            return x, states
        else:
            return x

In [33]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

#Uji Model

In [34]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In [35]:
model.summary()

Model: "my_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (64, 100, 256)              │          16,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru (GRU)                            │ ((64, 100, 1024), (64,      │       3,938,304 │
│                                      │ 1024))                      │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (64, 100, 66)               │          67,650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,022,850 (15.35 MB)

 Trainable params: 4,022,850 (15.35 MB)

 Non-trainable params: 0 (0.00 B)

In [36]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [37]:
sampled_indices

array([62, 58, 24, 12, 16, 20, 25, 53, 26, 34, 46, 51,  1, 18, 47, 56, 35,
       19, 41, 10, 55, 61,  2, 53, 14, 53, 41, 53, 59,  0, 59, 28, 33, 65,
        1, 31, 10, 35,  6, 13, 17, 45, 49, 37, 31, 18, 59, 33, 39, 36, 22,
        6, 58,  6, 38, 37, 63, 19,  5,  2, 10, 33, 64,  5, 18, 52, 55, 48,
       10, 45,  0, 53, 61, 43, 34, 18,  9, 48, 28, 32, 34, 27,  5, 15, 62,
       29, 49, 59, 55, 59, 12, 13, 65, 25, 45, 45, 52,  4, 46, 53])

In [39]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'ion, God he knows, not I,\nThe queen your mother, and your brother York,\nHave taken sanctuary: the te'

Next Char Predictions:
 b"wsK;CGLnMUgl\nEhqVFb3pv nAnbnt[UNK]tOTz\nR3V'?DfjXREtTZWI's'YXxF& 3Ty&Empi3f[UNK]nvdUE.iOSUN&BwPjtpt;?zLffm$gn"


#Train Model

In [41]:
#Tambahan optimizer dan fungsi loss
loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [42]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1897, shape=(), dtype=float32)


In [43]:
tf.exp(example_batch_mean_loss).numpy()

66.003

In [44]:
model.compile(optimizer='adam', loss=loss)

In [46]:
#Konfigurasi Checkpoints
import os

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [47]:
#Lakukan Proses Training
EPOCHS=20

In [49]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1031s 6s/step - loss: 3.0368
Epoch 2/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1004s 6s/step - loss: 1.9090
Epoch 3/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1051s 6s/step - loss: 1.6254
Epoch 4/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1051s 6s/step - loss: 1.4761
Epoch 5/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1045s 6s/step - loss: 1.3900
Epoch 6/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1034s 6s/step - loss: 1.3277
Epoch 7/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1038s 6s/step - loss: 1.2779
Epoch 8/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1047s 6s/step - loss: 1.2322
Epoch 9/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1035s 6s/step - loss: 1.1918
Epoch 10/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1047s 6s/step - loss: 1.1504
Epoch 11/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1040s 6s/step - loss: 1.1095
Epoch 12/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1055s 6s/step - loss: 1.0621
Epoch 13/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1041s 6s/step - loss: 1.0212
Epoch 14/20
172/172 ━━━━━━━━━━━━━━━━━━━━ 1014s 6s/step - loss: 0.9757
Epoch 15/20
172/172 ━━━━━━━━━

#Generate Teks

In [50]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "[UNK]" from being generated.
    skip_ids = self.ids_from_chars(['[UNK]'])[:, None]
    sparse_mask = tf.SparseTensor(
        # Put a -inf at each bad index.
        values=[-float('inf')]*len(skip_ids),
        indices=skip_ids,
        # Match the shape to the vocabulary
        dense_shape=[len(ids_from_chars.get_vocabulary())])
    self.prediction_mask = tf.sparse.to_dense(sparse_mask)

  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)
    # Only use the last prediction.
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "[UNK]" from being generated.
    predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1)

    # Convert from token ids to characters
    predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
    return predicted_chars, states

In [53]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

In [54]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

ROMEO:
I am agree in hair. These virtue is
Likelied Rome.

CURTIS:
Is there remember your office.

CAPULET:
O my lord, Warwick, bear him first to tell:
A fenter loud, we shall never saw so understand
'Redime't to repart this hard now was but
And replied.

POLIXENES:
No doubt, sure, is it perform; looking at him!
Give me that answer when I see, good night!

ROMEO:
O, let me almost cold with happy days!

GREMIO:
I am away! before you forgot to him.

PERDITA:
Now tell me that, and
thou shalt follow their business?

SICINIUS:
Pray you, good Sir Jupine.
Show me and rot and Edward is the nature of the
witness to say, bestow thy fire and loves me here:
Death is Warwick to the Threes, and twenty courtesiness,
Twenty years age in his arrival and sin
Could ever see my feigned honour.
These worth of waters caught do, thou diest:
How cetter our husband!

PROSPERO:
Doubt when All-Son's?
What's in a drum, that is home and happy kings,
Unless thou obans'd with the fires afford
And hanger than an unat

In [55]:
start = time.time()
states = None
next_char = tf.constant(['ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:', 'ROMEO:'])
result = [next_char]

for n in range(1000):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result, '\n\n' + '_'*80)
print('\nRun time:', end - start)

tf.Tensor(
[b"ROMEO:\nThat fool's factious date!\n\nMERCUTIO:\nO, where'er he fond, sir, well you do lie at mereit\nHath danced by the aim when 'tis kind of kings;\nOur scenter's naught eye shall miss hurdly king.\nO, how much less depate him for the right,\nNo stumble. Let us to me who loves' toeth\nare height for our fivers. My thoughts\nDo all fresh-pition foul interchange\nWill keep the wrong and dukedom and his thumb at us,\nOr children's son be breed from sunder,\nOr what it was at fubble,\nThe heaven finding bash with her in IVerly:\nTake him our attendance has my datele, he\nmost-villant-wounded, at Manife the air with light.\nSee you the next way, slaughter'd here shall\nnever sing, the heavens' ergosition\nOf thy debrices of the while!\nWhy, when I was counted-eleven abroad!\n\nKATHARINA:\nNo; thou art burnt.\n\nCAPULET:\nAs sorrow ends asided in King Edward's children?\n\nBUCKINGHAM:\nHere comes your mother and the regard of England;\nThere's seated fellow, respice the one m

#Ekspor Model Generator

In [56]:
tf.saved_model.save(one_step_model, 'one_step')
one_step_reloaded = tf.saved_model.load('one_step')

In [57]:
states = None
next_char = tf.constant(['ROMEO:'])
result = [next_char]

for n in range(100):
  next_char, states = one_step_reloaded.generate_one_step(next_char, states=states)
  result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

ROMEO:
A second nature, but only lead,
Let him be charged a fee, but of aukel,
Which weapons to entreat yo


#Tugas

In [58]:
import tensorflow as tf
import numpy as np
import os
import time

In [59]:
class CustomTraining(MyModel):
  @tf.function
  def train_step(self, inputs):
      inputs, labels = inputs
      with tf.GradientTape() as tape:
          predictions = self(inputs, training=True)
          loss = self.loss(labels, predictions)
      grads = tape.gradient(loss, model.trainable_variables)
      self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

      return {'loss': loss}

In [60]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [61]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [62]:
model.fit(dataset, epochs=1)

172/172 ━━━━━━━━━━━━━━━━━━━━ 1012s 6s/step - loss: 2.4929


In [63]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    # Memperbaiki kesalahan reset_state (tanpa "s")
    mean.reset_state()

    for (batch_n, (inp, target)) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs['loss'])

        if batch_n % 50 == 0:
            template = f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f'Epoch {epoch+1} Loss: {mean.result().numpy():.4f}')
    print(f'Time taken for 1 epoch {time.time() - start:.2f} sec')
    print("_"*80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 2.0236
Epoch 1 Batch 50 Loss 1.9041
Epoch 1 Batch 100 Loss 1.8118
Epoch 1 Batch 150 Loss 1.7349

Epoch 1 Loss: 1.8299
Time taken for 1 epoch 1041.91 sec
________________________________________________________________________________
Epoch 2 Batch 0 Loss 1.7066
Epoch 2 Batch 50 Loss 1.5966
Epoch 2 Batch 100 Loss 1.5901
Epoch 2 Batch 150 Loss 1.5020

Epoch 2 Loss: 1.5884
Time taken for 1 epoch 1023.25 sec
________________________________________________________________________________
Epoch 3 Batch 0 Loss 1.5271
Epoch 3 Batch 50 Loss 1.4360
Epoch 3 Batch 100 Loss 1.4630
Epoch 3 Batch 150 Loss 1.4437

Epoch 3 Loss: 1.4625
Time taken for 1 epoch 1010.31 sec
________________________________________________________________________________
Epoch 4 Batch 0 Loss 1.4522
Epoch 4 Batch 50 Loss 1.3620
Epoch 4 Batch 100 Loss 1.3980
Epoch 4 Batch 150 Loss 1.3684

Epoch 4 Loss: 1.3850
Time taken for 1 epoch 1010.29 sec
_____________________________________________________________